## 基本的数据处理

`Series` 和 `Dataframe` 是 `pandas` 库里面两个常见的数据类型，为了方便你们快速上手，我们会对它们进行相应的介绍以及进行一些简单的代码实操。

💡 如果你不会使用 `Jupyter Notebook`，请移步至：[Jupyter Notebook的使用](https://zhuanlan.zhihu.com/p/33105153)

#### Series


In [ ]:
import numpy as np
import pandas as pd

from pandas import Series, DataFrame

In [ ]:
#set seed to ensure that we can replicate the numbers generated (for random function)
np.random.seed(123)

In [ ]:
s = Series(np.random.randn(10))
s

In [ ]:
# You can use index to select specific values in s, have a try!
# s[1:3],s[[1,2]],s[2]

Series是一个具有轴标签的1-D numpy数组

In [ ]:
# getting the index
# Pandas data structure is extending numpy ndarray
# but it comes with index(es)
s.index

In [ ]:
# set index for the copy of s.
s1 = s.copy()
s1.index = ["item 0", "item 1", "item 2", "item 3", "item 4", "item 5", "item 6", "item 7", "item 8", "item 9"]
s1

In [ ]:
# create a reverse of s1
s2 = Series(s.values[::-1], index=["item 0", "item 1", "item 2", "item 3", "item 4", "item 5", "item 6", "item 7", "item 8", "item 9"])
s2

In [ ]:
# Try to perform s1 + s2
s1 + s2

In [ ]:
# notice we are having same indexes
s3 = pd.Series(["d", "e"])
s4 = pd.Series(["f", "g"])
s5 = pd.concat([s2, s3])
s5

In [ ]:
s5[1]

#### ❓Q1. 我们如何将 `s3` 和 `s4` 和合并在一起，同时保证 `index` 是递增的呢？

In [ ]:
s6=pd.concat([s3,s4],ignore_index=True)
s6

#### Dataframe

`DataFrame` 是一个二维表格数据。`Series` 可用的操作也可用于 `Dataframe` (或类似的方式)

In [ ]:
df1 = DataFrame([[1,2,3,4,5], [6,7,8,9,10]], columns=["a", "b", "c", "d", "e"])
df1

请下载我们为你准备的 `googleplaystore.csv` 文件，运用 `pandas` 库将其读入，并根据下面的要求进行相应的处理。  

In [ ]:
gplay = pd.read_csv("googleplaystore.csv")
gplay

#### ❓Q2. 试去除 `gplay` 中的 `Nan` 数据

In [ ]:
gplay.dropna(inplace=True)


gplay

#### ❓Q3. 查看`gplay`中重复的列，试使用`iloc`和`loc`来选中重复的行，并谈谈他们的区别

In [ ]:
#also possible to use the duplicated() method 
#(which will determine whether 2 rows are duplicated by checking all the column values)
duplicated_apps = gplay.loc[gplay.duplicated()]

print(gplay.duplicated())

#print out number of rows that is duplicated 
#by default it will keep the first row as not duplicated i.e. subsequent rows with the exact same column values
#will be treated as duplicated
print(len(duplicated_apps))

#this will return all the duplicated rows
duplicated_apps

In [ ]:
# Try to use loc
# duplicated_apps.loc[0]

# iloc version, select the first_dup_app in duplicated_apps
first_dup_app=duplicated_apps.iloc[0]

first_dup_app

In [ ]:
''' Your Understanding Here '''
#一开始我的理解是分别用iloc和loc来实现第二步选中，但是看来应该是分别用来实现两步？这里重复的“列”和"行“确实把我搞晕了，行实际上指的是横坐标？第一步明明
# 说了是检查duplicated rows（选中一行，即纵坐标的选中），为什么又说是查看gplay中重复的列？duplicated()返回的series中包含了所有重复行的名字，然后iloc
#【0】即表示按顺序第一条

In [ ]:
# like Series, we can use the drop_duplicates() to remove the duplicated rows
print(gplay.shape)
print(gplay.drop_duplicates().shape)

#### 读取数据

请下载我们为你准备的 `Auto.csv` 文件，运用 `pandas` 库将其读入，并根据下面的要求进行相应的处理。  

In [ ]:
df = pd.read_csv("Auto.csv")
df

In [ ]:
df.dtypes

注意，马力不是一个数字类型  
主要原因是数据集中有一些缺失的值，用“?”字符表示

还要注意，origin实际上是一个使用整数表示的分类变量，而不是实质上的数字变量  
还要注意，年份既可以是时间的度量，也可以是描述汽车版本的分类变量

#### ❓Q4. 阅读 `pd.read_csv()` 的文档，并将所有含有 `?` 的单元以 `Nan` 的形式读入

In [ ]:
df = pd.read_csv("Auto.csv", na_values="?")
df["origin"] = df["origin"].astype(object)
df["year"] = df["year"].astype(object)


# Hint: You may also need to change the origin and year into object. You may need astype() function.
# make sure you have replace all the ? cells on the **initial** data.
df.dtypes

In [ ]:
# this will get all the rows with at least a NaN value
# axis = 1 means "row-wise"
df.isna().any(axis=1)

In [ ]:
# rows with nan values
df[df.isna().any(axis=1)]

#### ❓Q5. 将 `df` 中的所有含有 `Nan` 的行全部移除

In [ ]:
df = df[~df.isna().any(axis=1)]
df
# Hint: Actually one line could solve this problem. Think about ~ and .any() function.

#### 数据可视化

在机器学习与深度学习的训练中，我们常常需要将一些损失或者其他的关键数据进行可视化，使得训练效果变得直观易于分析。

In [ ]:
import matplotlib.pyplot as plt
import statsmodels.api as sm

⬆️: 你可能没有上述的Python库，自己去安装它。

对于上面我们处理过的 `df`，我们尝试着使用最小二乘法(OLS)来预测 `mpg` 和 `horsepower` 两者之间的联系。

你可能不能理解下面的代码在干什么，但是我们希望你去运行一下这段代码来对于线性回归有一个初步的了解。

In [ ]:
X = df["horsepower"]
y = df["mpg"]
X = sm.add_constant(X)
model1 = sm.OLS(y,X).fit()


#model1 = sm.OLS(y,X.astype(float)).fit() 


#这段代码出现了报错ValueError: Pandas data cast to numpy dtype of object. Check input data with np.asarray(data).需要加上astype（）

#### ❓Q6. 了解 `matplotlib`，试对上述的代码的预测结果进行可视化处理

In [ ]:
plt.scatter(X['horsepower'], y, label='Actual')
plt.plot(X['horsepower'], model1.predict(X), label='Predicted', color='red')
plt.xlabel('Horsepower')
plt.ylabel('MPG')
plt.legend()
plt.show()
#我感觉这有些难，画出来的图像我没法弄成有效的
# Hint: Consider about function scatter() and plot() in matplotlib.pyplot. 
# You may use model1.predict(X) to get the predicted value.

#### 机器学习初探

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

请下载我们为你准备的 `email_spam.csv` 文件，运用 `pandas` 库将其读入，并根据下面的要求进行相应的处理。  

In [ ]:
df = pd.read_csv("email_spam.csv")
df

#### ❓Q7. 现在我们将考虑对垃圾邮件进行分类。但在生成模型之前，首先将所有列数据转换为数字。具体地说:
- `no` 修改为 `0`，`yes` 修改为 `1`
- 为 `format` 和 `number` 使用虚拟变量编码(你可以使用 `pd.get_dummies()` 来生成虚拟变量)

In [ ]:
def map_values(value):
    if value == 'no':
        return 0
    elif value == 'yes':
        return 1
    else:
        return value
df = df.applymap(map_values)
df = pd.get_dummies(df, columns=['format', 'number'], drop_first=True)
X = df.iloc[:,1:]
y = df["spam"]
# Hint: While you are using pd.get_dummies(), you may need to drop the previous column.

#### ❓Q8. 使用 `train_test_split()` 将数据集划分为`70%`的训练集和`30%`的测试集(设置 `random_test=123` 以确保我们可以复制分割)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=123)

#### ❓Q9. 使用逻辑回归的模型来进行预测，同时看看你的准确率如何

In [ ]:

model = LogisticRegression()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
#我没办法了解这些太高级的api的细节，只能从网上查找代码，但是我保证这些代码都从头到尾跑通过了，曾经处理过错误
# Hint: While you are evaluating the model, you may need to use test data.
# Think about the work you have done in the previous question. 

y_pred_score = accuracy_score(y_test,y_pred)
y_pred_score

#### ❓Q10. 由于数据是不平衡的，最好生成混淆矩阵。了解什么是混淆矩阵以及如何在sklearn上实现它

In [ ]:
''' If you are interested in the confusion matrix, you may use the following code to get the confusion matrix. '''

In [ ]:
# You don't need to understand the code below.
# It is just for plotting the confusion matrix.
from sklearn.metrics import confusion_matrix,ConfusionMatrixDisplay
confusion_matrix(y_test, y_pred)
ConfusionMatrixDisplay(confusion_matrix(y_test,y_pred)).plot()